In [1]:
import json
import re

import pandas as pd

In [2]:
PATH = 'raw/Pflegestatistik_2013_Hessen.xls'
data = []

In [3]:
counties = json.load(open('../geodata/landkreise.geojson'))
counties = [f['properties'] for f in counties['features']]
county_mapping = {x['GEN']: x['RS'] for x in counties}
county_mapping['Kassel, St.'] = '06611'


In [4]:
def get_ags_for_name(name):
    if name in ('key', 'Hessen') or 'Reg.Bez' in name:
        return None
    name = name.replace('-\n', '')
    ags = county_mapping.get(name)
    if ags is None:
        print(name)
    return ags


def get_column_values(df, mapping, prefix):
    for c in df.columns:
        ags = get_ags_for_name(c)
        if ags is None:
            continue

        for k, v in mapping.items():
            val = df[df['key'].str.contains(k, regex=False).fillna(False)].iloc[0][c]
            if val == '–':
                val = 0.0
            yield ags, prefix, v, float(val)

In [5]:
df = pd.read_excel(PATH, sheetname='Vergütung', skiprows=2)
df = df.rename(columns={
    'Art der Vergütung / Region': 'key'
})

cost_mapping = {
    'Pflegeklasse 1': 'costs_nursing_class_1',
    'Pflegeklasse 2': 'costs_nursing_class_2',
    'Pflegeklasse 3': 'costs_nursing_class_3',
    'Verpflegung': 'food',
}

data.extend(list(get_column_values(df, cost_mapping, 'costs')))
df.head()

,key,Hessen,Reg.Bez. Darmstadt,Darmstadt,Frankfurt am Main,Offenbach am Main,Wiesbaden,Bergstraße,Darmstadt-Dieburg,Groß-Gerau,...,Marburg-Biedenkopf,Vogelsberg- kreis,Reg.Bez. Kassel,"Kassel, St.",Fulda,Hersfeld-Rotenburg,Kassel,Schwalm-Eder-Kreis,Waldeck-Frankenberg,Werra-Meißner-Kreis
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,Durchschnittliche Vergütungen insgesamt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df = pd.read_excel(PATH, sheetname='Verfügbare Plätze', skiprows=2)
df = df.rename(columns={
    'Art der Verfügbaren Plätze / Region': 'key'
})

bed_mapping = {
  '1-Bett-Zimmern': '1-bed',
  '2-Bett-Zimmern': '2-bed',
  '3-Bett-Zimmern': '3-bed',
  '4 und mehr-Bett-Zimmern': '4-bed'
}
data.extend(list(get_column_values(df, bed_mapping, 'beds')))
df.head()

,key,Hessen,Reg.Bez. Darmstadt,Darmstadt,Frankfurt am Main,Offenbach am Main,Wiesbaden,Bergstraße,Darmstadt-Dieburg,Groß-Gerau,...,Marburg-Biedenkopf,Vogelsberg- kreis,Reg.Bez. Kassel,"Kassel, St.",Fulda,Hersfeld-Rotenburg,Kassel,Schwalm-Eder-Kreis,Waldeck-Frankenberg,Werra-Meißner-Kreis
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,Verfügbare Plätze insgesamt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df = pd.read_excel(PATH, sheetname='Pers', skiprows=2)
df = df.rename(columns={
    'Personalangaben / Region': 'key'
})

personal_mapping = {
    'Vollzeitbeschäftigt': 'fully_employed',
    'Teilzeitbeschäftigt': 'part-time',
    'Pflege und Betreuung': 'nursing_and_care',
    'soziale Betreuung': 'social_care',
    'zusätzliche Betreuung (§ 87b SGB XI)': 'other_care',
    'Personal insgesamt': 'total_personnel',
    'Auszubildende/-r, (Um-)Schüler/-in': 'trainee',
    'staatlich anerkannte/-r Altenpfleger/-in': 'state_certified',
    'Gesundheits- und Krankenpfleger/-in': 'nurse',
    'Gesundheits- und Kinderkrankenpfleger/-in': 'nurse_children',
}

data.extend(list(get_column_values(df, personal_mapping, 'personal')))
df.head()

,key,Hessen,Reg.Bez. Darmstadt,Darmstadt,Frankfurt am Main,Offenbach am Main,Wiesbaden,Bergstraße,Darmstadt-Dieburg,Groß-Gerau,...,Marburg-Biedenkopf,Vogelsberg- kreis,Reg.Bez. Kassel,"Kassel, St.",Fulda,Hersfeld-Rotenburg,Kassel,Schwalm-Eder-Kreis,Waldeck-Frankenberg,Werra-Meißner-Kreis
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,Personal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
df = pd.read_excel(PATH, sheetname='Pflegebeduerftige', skiprows=2)
df = df.rename(columns={
    'Personalangaben / Region': 'key'
})

recipient_mapping = {
    'Pflegestufe I ': 'recipients_nursing_class_1',
    'Pflegestufe II ': 'recipients_nursing_class_2',
    'Pflegestufe III ': 'recipients_nursing_class_3',
    'Pflegestufe zugeordnet': 'recipients_nursing_class_unknown',
}


data.extend(list(get_column_values(df, recipient_mapping, 'recipients')))
df.head()

,key,Hessen,Reg.Bez. Darmstadt,Darmstadt,Frankfurt am Main,Offenbach am Main,Wiesbaden,Bergstraße,Darmstadt-Dieburg,Groß-Gerau,...,Marburg-Biedenkopf,Vogelsberg- kreis,Reg.Bez. Kassel,"Kassel, St.",Fulda,Hersfeld-Rotenburg,Kassel,Schwalm-Eder-Kreis,Waldeck-Frankenberg,Werra-Meißner-Kreis
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,Pflegebedürftige,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df = pd.DataFrame(data)
df = df.rename(columns={
    0: 'ags',
    1: 'type',
    2: 'key',
    3: 'value'
})
df['state'] = 'Hessen'
df.to_csv('csvs/hessen.csv', index=False)
df.head()

,ags,type,key,value,state
0,06411,costs,costs_nursing_class_3,83.78,Hessen
1,06411,costs,costs_nursing_class_2,65.43,Hessen
2,06411,costs,costs_nursing_class_1,48.29,Hessen
3,06411,costs,food,19.65,Hessen
4,06412,costs,costs_nursing_class_3,85.58,Hessen


In [10]:
df[(df['type'] == 'personal') & (df['ags'] == '06411')]

,ags,type,key,value,state
208,06411,personal,nurse_children,12.0,Hessen
209,06411,personal,trainee,51.0,Hessen
210,06411,personal,nurse,86.0,Hessen
211,06411,personal,other_care,35.0,Hessen
212,06411,personal,fully_employed,412.0,Hessen
213,06411,personal,total_personnel,1096.0,Hessen
214,06411,personal,state_certified,280.0,Hessen
215,06411,personal,social_care,51.0,Hessen
216,06411,personal,nursing_and_care,711.0,Hessen
217,06411,personal,part-time,620.0,Hessen
